## Data Cleaning and Preparation

In [1]:
import numpy as np
import pandas as pd

## Handling Missing Data

For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data. We call this a sentinel value that can be easily detected:

In [2]:
string_data = pd.Series(["aardvark", "artichoke", np.nan, "avocado"])

In [3]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

The built-in Python **None** value is also treated as NA in object arrays:

In [5]:
string_data[0] = None

In [6]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

* `dropna` Filter axis labels based on whether values for each label have missing data, with varying thresholds for how much missing data to tolerate.

* `fillna` Fill in missing data with some value or using an interpolation method such as 'ffill' or 'bfill'.

* `isnull` Return boolean values indicating which values are missing/NA.

* `notnull` Negation of `isnull`.

### Filtering Out Missing Data

`dropna` returns the Series with only the non-null data and index values:

In [7]:
from numpy import nan as NA

In [8]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [9]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [10]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows or columns that are all NA or only those containing any NAs. *`dropna` by default drops any row containing a missing value:

In [11]:
data = pd.DataFrame([[1.0, 6.5, 3.0], [1.0, NA, NA], [NA, NA, NA], [NA, 6.5, 3.0]])

In [12]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [13]:
cleaned = data.dropna()

In [14]:
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing `how='all'` will only drop rows that are all NA:

In [15]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass `axis=1`:

In [16]:
data[4] = NA

In [17]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [18]:
data.dropna(axis=1, how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [19]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [20]:
data.dropna(how="any")

,0,1,2,4


Suppose you want to keep only rows containing a certain number of observations. You can indicate this with the `thresh` argument:

In [48]:
df = pd.DataFrame(np.arange(21).reshape(7, 3))

In [49]:
df.iloc[:4, 1] = NA

In [50]:
df.iloc[:2, 2] = NA

In [51]:
df

,0,1,2
0,0,NaN,NaN
1,3,NaN,NaN
2,6,NaN,8.0
3,9,NaN,11.0
4,12,13.0,14.0
5,15,16.0,17.0
6,18,19.0,20.0


In [52]:
df.dropna()

,0,1,2
4,12,13.0,14.0
5,15,16.0,17.0
6,18,19.0,20.0


In [53]:
df.dropna(axis=1)

,0
0,0
1,3
2,6
3,9
4,12
5,15
6,18


In [54]:
df.dropna(thresh=2)

,0,1,2
2,6,NaN,8.0
3,9,NaN,11.0
4,12,13.0,14.0
5,15,16.0,17.0
6,18,19.0,20.0


### Filling In Missing Data

Calling `fillna` with a constant replaces missing values with that value:

In [55]:
df.fillna(0)

,0,1,2
0,0,0.0,0.0
1,3,0.0,0.0
2,6,0.0,8.0
3,9,0.0,11.0
4,12,13.0,14.0
5,15,16.0,17.0
6,18,19.0,20.0


Calling `fillna` with a dict, you can use a different fill value for each column:

In [56]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,0,0.5,0.0
1,3,0.5,0.0
2,6,0.5,8.0
3,9,0.5,11.0
4,12,13.0,14.0
5,15,16.0,17.0
6,18,19.0,20.0


`fillna` returns a new object, but you can modify the existing object in-place:

In [57]:
df.fillna(0, inplace=True)

In [58]:
df

,0,1,2
0,0,0.0,0.0
1,3,0.0,0.0
2,6,0.0,8.0
3,9,0.0,11.0
4,12,13.0,14.0
5,15,16.0,17.0
6,18,19.0,20.0


The same interpolation methods available for reindexing can be used with `fillna`:

In [60]:
df = pd.DataFrame(np.arange(18).reshape((6, 3)))

In [61]:
df.iloc[2:, 1] = NA

In [62]:
df.iloc[4:, 2] = NA

In [63]:
df

,0,1,2
0,0,1.0,2.0
1,3,4.0,5.0
2,6,NaN,8.0
3,9,NaN,11.0
4,12,NaN,NaN
5,15,NaN,NaN


In [64]:
df.fillna(method="ffill")

,0,1,2
0,0,1.0,2.0
1,3,4.0,5.0
2,6,4.0,8.0
3,9,4.0,11.0
4,12,4.0,11.0
5,15,4.0,11.0


In [65]:
df.fillna(method="ffill", limit=2)

,0,1,2
0,0,1.0,2.0
1,3,4.0,5.0
2,6,4.0,8.0
3,9,4.0,11.0
4,12,NaN,11.0
5,15,NaN,11.0


With `fillna` you can do lots of other things with a little creativity. For example, you might pass the mean or median value of a Series:

In [66]:
data = pd.Series([1.0, NA, 3.5, NA, 7])

In [67]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Data Transformation

### Removing Duplicates

The DataFrame method `duplicated` returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not:

In [68]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"], "k2": [1, 1, 2, 3, 3, 4, 4]})

In [69]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [70]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

`drop_duplicates` returns a DataFrame where the duplicated array is *False*:

In [71]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider *all of the columns*; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column:

In [72]:
data["v1"] = range(7)

In [73]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [74]:
data.drop_duplicates(["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` and `drop_duplicates` by default keep the first observed value combination. Passing `keep='last'` will return the last one:

In [75]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [76]:
data.drop_duplicates(["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

You may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:

In [77]:
data = pd.DataFrame(
    {
        "food": [
            "bacon",
            "pulled pork",
            "bacon",
            "Pastrami",
            "corned beef",
            "Bacon",
            "pastrami",
            "honey ham",
            "nova lox",
        ],
        "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6],
    }
)

In [78]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food came from. Let’s write down a mapping of each distinct meat type to the kind of animal:

In [79]:
meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow",
    "corned beef": "cow",
    "honey ham": "pig",
    "nova lox": "salmon",
}

The `map` method on a Series accepts a function or dict-like object containing a mapping, but here we have a small problem in that some of the meats are capitalized and others are not. Thus, we need to convert each value to lowercase using the `str.lower` Series method:

In [80]:
lowercased = data["food"].str.lower()

In [81]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [82]:
lowercased.map(meat_to_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [83]:
data["animal"] = lowercased.map(meat_to_animal)

In [84]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all the work:

In [85]:
data["food"].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

**Using map is a convenient way to perform element-wise transformations**

In [86]:
f = lambda x: meat_to_animal[x.lower()]

In [87]:
f("pastrami")

'cow'

### Replacing Values

Filling in missing data with the `fillna` method is a special case of more general value replacement. As you've already seen, `map` can be used to modify a subset of values in an object but `replace` provides a simpler and more flexible way to do so. Let's consider this Series:

In [88]:
data = pd.Series([1.0, -999.0, 2.0, -999.0, -1000.0, 3.0])

In [89]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be sentinel values for missing data. To replace these with NA values that pandas understands, we can use replace, producing a new Series (unless you pass `inplace=True`):

In [90]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the substitute value:

In [91]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes:

In [92]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dict:

In [93]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. You can also modify the axes in-place without creating a new data structure. Here’s a simple example:

In [94]:
data = pd.DataFrame(
    np.arange(12).reshape((3, 4)),
    index=["Ohio", "Colorado", "New York"],
    columns=["one", "two", "three", "four"],
)

In [95]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Like a Series, the axis indexes have a map method:

In [96]:
transform = lambda x: x[:4].upper()

In [97]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [98]:
data.index = data.index.map(transform)

In [99]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


If you want to create a transformed version of a dataset without modifying the original, a useful method is `rename`:

In [101]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Notably, `rename` can be used in conjunction with a dict-like object providing new values for a subset of the axis labels:

In [103]:
data.rename(index={"OHIO": "INDIANA"}, columns={"three": "peekaboo"})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


`rename` saves you from the chore of copying the DataFrame manually and assigning to its index and columns attributes.

### Discretization and Binning

Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [104]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let's divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do so, you have to use `cut`, a function in pandas:

In [105]:
bins = [18, 25, 35, 60, 100]

In [106]:
cats = pd.cut(ages, bins)

In [107]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special `Categorical` object. The output you see describes the bins computed by `pandas.cut`. You can treat it like an array of strings indicating the bin name; internally it contains a `categories` array specifying the distinct category names along with a labeling for the ages data in the `codes` attribute:

In [108]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [109]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [110]:
len(cats)

12

In [111]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

Consistent with mathematical notation for intervals, a parenthesis means that the side
is _open_, while the square bracket means it is _closed_ (inclusive). You can change which side is closed by passing `right=False`:

In [112]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

You can also pass your own bin names by passing a list or array to the `labels` option:

In [113]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]

In [114]:
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to `cut` instead of explicit bin edges, it will compute **equal-length** (same distance between edges) bins based on the minimum and maximum values in the data.
Consider the case of some uniformly distributed data chopped into fourths:

In [115]:
data = np.random.rand(20)

In [116]:
pd.cut(data, 4, precision=2)

[(0.75, 0.99], (0.51, 0.75], (0.024, 0.27], (0.27, 0.51], (0.024, 0.27], ..., (0.024, 0.27], (0.27, 0.51], (0.024, 0.27], (0.51, 0.75], (0.75, 0.99]]
Length: 20
Categories (4, interval[float64, right]): [(0.024, 0.27] < (0.27, 0.51] < (0.51, 0.75] < (0.75, 0.99]]

A closely related function, `qcut`, bins the data based on sample quantiles. Depending on the distribution of the data, using `cut` will not usually result in each bin having the same number of data points. 
Since `qcut` uses sample quantiles instead, by definition you will obtain roughly **equal-size** (same bin sizes) bins:

In [119]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles

In [120]:
cats

[(0.00715, 0.666], (-0.609, 0.00715], (-0.609, 0.00715], (0.666, 2.909], (-0.609, 0.00715], ..., (0.666, 2.909], (0.00715, 0.666], (-3.2849999999999997, -0.609], (-3.2849999999999997, -0.609], (-3.2849999999999997, -0.609]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.2849999999999997, -0.609] < (-0.609, 0.00715] < (0.00715, 0.666] < (0.666, 2.909]]

Note that quantiles is just the most general term for things like percentiles, quartiles, and medians. When we ask for quintiles with` qcut`, the bins will be chosen so that we have the *same number of records in each bin*. If we have 30 records, we will have 6 in each bin (the output should look like the following, although the breakpoints will differ due to the random draw):

```python
pd.qcut(factors, 5).value_counts()

[-2.578, -0.829]    6
(-0.829, -0.36]     6
(-0.36, 0.366]      6
(0.366, 0.868]      6
(0.868, 2.617]      6
```

Conversely, for `cut` we will see something more uneven:

```python
pd.cut(factors, 5).value_counts()

(-2.583, -1.539]    5
(-1.539, -0.5]      5
(-0.5, 0.539]       9
(0.539, 1.578]      9
(1.578, 2.617]      2
``` 
 

In [121]:
pd.value_counts(cats)

(-3.2849999999999997, -0.609]    250
(-0.609, 0.00715]                250
(0.00715, 0.666]                 250
(0.666, 2.909]                   250
dtype: int64

Similar to `cut` you can pass your own quantiles (numbers between 0 and 1, inclusive):

In [122]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.0])

[(0.00715, 1.215], (-1.206, 0.00715], (-1.206, 0.00715], (0.00715, 1.215], (-1.206, 0.00715], ..., (0.00715, 1.215], (0.00715, 1.215], (-1.206, 0.00715], (-1.206, 0.00715], (-3.2849999999999997, -1.206]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.2849999999999997, -1.206] < (-1.206, 0.00715] < (0.00715, 1.215] < (1.215, 2.909]]

#### Detecting and Filtering Outliers

Consider a DataFrame with some normally distributed data:

In [123]:
data = pd.DataFrame(np.random.randn(1000, 4))

In [124]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.046042,0.040388,-0.021629,0.000965
std,1.018415,0.990515,1.010292,0.992723
min,-3.801045,-2.696462,-3.725141,-3.504824
25%,-0.629528,-0.689699,-0.736627,-0.656595
50%,0.050463,0.025748,0.000968,-0.003237
75%,0.729683,0.734755,0.678400,0.677671
max,3.353090,3.854725,3.045578,3.128736


Suppose you wanted to find values in one of the columns exceeding 3 in absolute value:

In [125]:
col = data[2]

In [126]:
col[np.abs(col) > 3]

62    -3.725141
536    3.045578
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or –3, you can use the **any** method on a boolean DataFrame:

In [127]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
62,0.105925,-0.061184,-3.725141,0.024281
173,-0.061947,-0.104432,0.772114,3.128736
216,-3.378441,-0.168534,-0.575253,0.314336
254,-1.391075,1.036128,-0.349519,-3.504824
536,-0.232001,0.725988,3.045578,-0.152170
754,3.353090,2.094373,-0.533570,0.755358
757,-3.801045,1.859047,-1.214971,0.423280
896,-0.129583,3.854725,-0.758065,0.335491
921,-0.377859,3.173847,1.853745,-0.483169


In [128]:
pd.options.display.max_rows = 10

In [129]:
(np.abs(data) > 3)

,0,1,2,3
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
995,False,False,False,False
996,False,False,False,False
997,False,False,False,False
998,False,False,False,False


In [130]:
(np.abs(data) > 3).any()  # is there any >3 in each column

0    True
1    True
2    True
3    True
dtype: bool

In [131]:
(np.abs(data) > 3).any(1)  # is there any >3 in each row

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3:

In [132]:
data[np.abs(data) > 3] = np.sign(data) * 3

In [133]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.046868,0.039360,-0.020949,0.001341
std,1.013399,0.987047,1.007751,0.990668
min,-3.000000,-2.696462,-3.000000,-3.000000
25%,-0.629528,-0.689699,-0.736627,-0.656595
50%,0.050463,0.025748,0.000968,-0.003237
75%,0.729683,0.734755,0.678400,0.677671
max,3.000000,3.000000,3.000000,3.000000


The statement `np.sign(data)` produces 1 and -1 values based on whether the values in *data* are positive or negative:

In [134]:
np.sign(data)

,0,1,2,3
0,-1.0,-1.0,-1.0,1.0
1,1.0,-1.0,-1.0,1.0
2,1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,1.0
4,-1.0,-1.0,-1.0,1.0
...,...,...,...,...
995,-1.0,1.0,1.0,-1.0
996,-1.0,-1.0,1.0,-1.0
997,-1.0,1.0,1.0,1.0
998,-1.0,-1.0,-1.0,-1.0


### Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the `numpy.random.permutation` function. Calling `permutation` with the length of the axis you want to permute produces an array of integers indicating the new ordering:

In [135]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))

In [136]:
sampler = np.random.permutation(5)

In [137]:
sampler

array([2, 4, 3, 0, 1])

That array can then be used in `iloc`-based indexing or the equivalent take function:

In [138]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [139]:
df.take(sampler)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
3,12,13,14,15
0,0,1,2,3
1,4,5,6,7


To select a random subset without replacement, you can use the `sample` method on Series and DataFrame:

In [140]:
df.sample(n=3)

,0,1,2,3
4,16,17,18,19
2,8,9,10,11
1,4,5,6,7


In [141]:
df.sample(n=3, replace=True)

,0,1,2,3
4,16,17,18,19
1,4,5,6,7
4,16,17,18,19


To generate a sample with replacement (to allow repeat choices), pass `replace=True` to sample:

In [142]:
choices = pd.Series([5, 7, -1, 6, 4])

In [143]:
draws = choices.sample(n=10, replace=True)

In [144]:
draws

3    6
4    4
2   -1
3    6
3    6
0    5
0    5
0    5
3    6
3    6
dtype: int64

### Computing Indicator/Dummy Variables

If a column in a DataFrame has k distinct values, you would derive a matrix or DataFrame with k columns containing all 1's and 0's. pandas has a `get_dummies` function for doing this, though devising one yourself is not difficult. Let's return to an earlier example DataFrame:

In [146]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], "data1": range(6)})

In [147]:
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [148]:
pd.get_dummies(df["key"])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In some cases, you may want to add a prefix to the columns in the indicator DataFrame, which can then be merged with the other data. `get_dummies` has a `prefix` argument for doing this:

In [149]:
dummies = pd.get_dummies(df["key"], prefix="key")

In [150]:
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [151]:
df_with_dummy = df[["data1"]].join(dummies)

In [152]:
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


MovieLens dataset Example

In [161]:
mnames = ["movie_id", "title", "genres"]

In [165]:
movies = pd.read_table(
    "data/movies.dat",
    engine="python",
    encoding="latin-1",
    sep="::",
    header=None,
    names=mnames,
)

In [166]:
movies

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


Adding indicator variables for each genre requires a little bit of wrangling. First, we extract the list of unique genres in the dataset:

In [167]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split("|"))
genres = pd.unique(all_genres)

In [168]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

One way to construct the indicator DataFrame is to start with a DataFrame of all zeros:

In [169]:
zero_matrix = np.zeros((len(movies), len(genres)))

In [170]:
dummies = pd.DataFrame(zero_matrix, columns=genres)

Now, iterate through each movie and set entries in each row of dummies to 1. To do this, we use the `dummies.columns` to compute the column indices for each genre:

In [171]:
gen = movies.genres[0]

In [172]:
gen.split("|")

['Animation', "Children's", 'Comedy']

In [173]:
dummies.columns.get_indexer(gen.split("|"))

array([0, 1, 2])

Then, we can use `.iloc` to set values based on these indices:

In [174]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split("|"))
    dummies.iloc[i, indices] = 1

In [175]:
movies_windic = movies.join(dummies.add_prefix("Genre_"))

In [176]:
movies_windic.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genre_Animation                             1.0
Genre_Children's                            1.0
                               ...             
Genre_War                                   0.0
Genre_Musical                               0.0
Genre_Mystery                               0.0
Genre_Film-Noir                             0.0
Genre_Western                               0.0
Name: 0, Length: 21, dtype: object

For much larger data, this method of constructing indicator variables with multiple membership is not especially speedy. It would be better to write a lower-level function that writes directly to a NumPy array, and then wrap the result in a DataFrame.

A useful recipe for statistical applications is to combine `get_dummies` with a discretization function like `cut`:

In [177]:
values = np.random.rand(10)

In [178]:
values

array([0.13201039, 0.46463459, 0.98969847, 0.71752538, 0.884624  ,
       0.16266861, 0.43692243, 0.29819296, 0.63450293, 0.90792004])

In [179]:
np.random.seed(12345)
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [180]:
pd.cut(values, bins)

[(0.0, 0.2], (0.4, 0.6], (0.8, 1.0], (0.6, 0.8], (0.8, 1.0], (0.0, 0.2], (0.4, 0.6], (0.2, 0.4], (0.6, 0.8], (0.8, 1.0]]
Categories (5, interval[float64, right]): [(0.0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1.0]]

In [181]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,1,0,0,0,0
1,0,0,1,0,0
2,0,0,0,0,1
3,0,0,0,1,0
4,0,0,0,0,1
5,1,0,0,0,0
6,0,0,1,0,0
7,0,1,0,0,0
8,0,0,0,1,0
9,0,0,0,0,1


## String Manipulation

Most text operations are made simple with the string object’s built-in methods. For more complex pattern matching and text manipulations, regular expressions may be needed. pandas adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data, additionally handling the annoyance of missing data.

### String Object Methods

In [182]:
val = "a,b, guido"

In [183]:
val.split(",")

['a', 'b', ' guido']

`split` is often combined with `strip` to trim whitespace (including line breaks):

In [184]:
pieces = [x.strip() for x in val.split(",")]

In [185]:
pieces

['a', 'b', 'guido']

In [186]:
first, second, third = pieces

In [187]:
first + "::" + second + "::" + third

'a::b::guido'

In [188]:
"::".join(pieces)

'a::b::guido'

Other methods are concerned with locating substrings. Using Python’s `in` keyword is the best way to detect a substring, though `index` and `find` can also be used:

In [189]:
"guido" in val

True

In [190]:
val.index(",")

1

In [191]:
val.find(",")

1

In [192]:
val.find(":")

-1

In [193]:
val.index(":")

ValueError: substring not found

`count` returns the number of occurrences of a particular substring:

In [194]:
val.count(",")

2

`replace` will substitute occurrences of one pattern for another. It is commonly used to delete patterns, too, by passing an empty string:

In [195]:
val.replace(",", "::")

'a::b:: guido'

In [197]:
val.replace(",", "")

'ab guido'

![alt text](images/stringmethods.png "Python built-in string methods")

### Regular Expressions

Regular expressions provide a flexible way to search or match (often more complex) string patterns in text. A single expression, commonly called a *regex*, is a string formed according to the regular expression language. Python’s built-in `re` module is responsible for applying regular expressions to strings.

The `re` module functions fall into three categories: *pattern matching, substitution, and splitting*.

Suppose we wanted to split a string with a variable number of whitespace characters (tabs, spaces, and newlines). The regex describing one or more whitespace characters is _\s+_:

In [198]:
import re

In [199]:
text = "foo bar\t baz \tqux"

In [200]:
re.split("\s+", text)

['foo', 'bar', 'baz', 'qux']

When you call `re.split('\s+', text)`, the regular expression is first *compiled,* and then its split method is called on the passed text. You can compile the regex yourself with `re.compile`, forming a reusable regex object

In [201]:
regex = re.compile("\s+")

In [202]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

If, instead, you wanted to get a list of all patterns matching the regex, you can use the `findall` method:

In [203]:
regex.findall(text)

[' ', '\t ', ' \t']

**avoid unwanted escaping with \ in a regular expression, use raw tring literals like r'C:\x' instead of the equivalent 'C:\\x'.**

**Creating a regex object with `re.compile` is highly recommended if you intend to apply the same expression to many strings; doing so will save CPU cycles.**

`match` and `search` are closely related to findall. 

While `findall` returns *all* matches in a string, `search` returns _only the first match_. `match` only matches at the beginning of the string.

let’s consider a block of text and a regular expression capable of identifying most email addresses:

In [204]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

In [205]:
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

In [206]:
# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

Using `findall` on the text produces a list of the email addresses:

In [207]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

`search` returns a special match object for the first email address in the text. For the preceding regex, the match object can only tell us the start and end position of the pattern in the string:

In [208]:
m = regex.search(text)

In [209]:
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [210]:
m.start()

5

In [211]:
m.end()

20

In [212]:
text[m.start() : m.end()]

'dave@google.com'

`regex.match` returns None, as it only will match if the pattern occurs at the start of the string:

In [213]:
print(regex.match(text))

None


`sub` will return a new string with occurrences of the pattern replaced by the a new string:

In [214]:
print(regex.sub("REDACTED", text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [215]:
re.sub(pattern, "REDACTED", text, flags=re.IGNORECASE)

'Dave REDACTED\nSteve REDACTED\nRob REDACTED\nRyan REDACTED\n'

Suppose you wanted to find email addresses and simultaneously segment each address into its three components: username, domain name, and domain suffix. To do this, put parentheses around the parts of the pattern to segment:

In [216]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"

In [217]:
regex = re.compile(pattern, flags=re.IGNORECASE)

A `match` object produced by this modified regex returns a tuple of the pattern components with its `groups` method:

In [218]:
m = regex.match("wesm@bright.net")

In [219]:
m.groups()

('wesm', 'bright', 'net')

`findall` returns a list of tuples when the pattern has groups:

In [220]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [221]:
for name, domain, suffix in regex.findall(text):
    print("Name: %s, Domain: %s, Suffix: %s" % (name, domain, suffix))

Name: dave, Domain: google, Suffix: com
Name: steve, Domain: gmail, Suffix: com
Name: rob, Domain: gmail, Suffix: com
Name: ryan, Domain: yahoo, Suffix: com


`sub` also has access to groups in each match using special symbols like \1 and \2. The symbol \1 corresponds to the first matched group, \2 corresponds to the second, and so forth:

In [222]:
print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



![alt text](images/regexs.png "Regular expression methods")

### Vectorized String Functions in pandas

In [223]:
data = {
    "Dave": "dave@google.com",
    "Steve": "steve@gmail.com",
    "Rob": "rob@gmail.com",
    "Wes": np.nan,
}

In [224]:
data = pd.Series(data)

In [225]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [226]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

String and regular expression methods can be applied (passing a _lambda_ or other function) to each value using **.map**, but it will fail on the NA (null) values. To cope with this, Series has array-oriented methods for string operations that skip NA values. These are accessed through Series's `.str` attribute; for example, we could check whether each email address has 'gmail' in it with `str.contains`:

In [227]:
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Regular expressions can be used, too, along with any *re* options like IGNORECASE

In [228]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [229]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

There are a couple of ways to do vectorized element retrieval. Either use `str.get` or index into the `str` attribute:

In [230]:
matches = data.str.match(pattern, flags=re.IGNORECASE)

In [231]:
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

To access elements in the embedded lists, we can pass an index to either of these functions:

In [235]:
data.str.get(1)

Dave       a
Steve      t
Rob        o
Wes      NaN
dtype: object

In [236]:
data.str[0]

Dave       d
Steve      s
Rob        r
Wes      NaN
dtype: object

You can similarly slice strings using this syntax:

In [237]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [238]:
data.str.len()

Dave     15.0
Steve    15.0
Rob      13.0
Wes       NaN
dtype: float64

![alt text](images/vectorizedstrings.png "Partial listing of vectorized string methods")